# Imports and choosing area of interest

In [1]:
# Imports
import ee
import pprint
import folium

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [3]:
# Get an area in Brazil using http://geojson.io
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -50.69091796875,
              -9.96885060854611
            ],
            [
              -50.614013671875,
              -9.96885060854611
            ],
            [
              -50.614013671875,
              -9.893098633379571
            ],
            [
              -50.69091796875,
              -9.893098633379571
            ],
            [
              -50.69091796875,
              -9.96885060854611
            ]
          ]
        ]
      }
    }
  ]
}

# Put coordinates into ee format
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)



# Loading Vancutsem and Landsat data

In [39]:
# Vancutsem data guide available at https://forobs.jrc.ec.europa.eu/TMF/download/TMF_DataUsersGuide.pdf

# the year a pixel is first deforested
deforestation_year = ee.ImageCollection('projects/JRC/TMF/v1_2020/DeforestationYear').select("DeforestationYear").filterBounds(aoi)#.filterDate('2014-01-01', '2020-01-01')

# total number of disturbances
intensity = ee.ImageCollection('projects/JRC/TMF/v1_2020/Intensity').select("Intensity").filterBounds(aoi)#.filterDate('2014-01-01', '2020-01-01')

# transition map: which class each pixel is in
transition_map = ee.ImageCollection('projects/JRC/TMF/v1_2020/TransitionMap_MainClasses').filterBounds(aoi).mosaic()#.filterDate('2014-01-01', '2020-01-01').select("TransitionMap_MainClasses")

# PALSAR SAR data
palsar = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR').filterBounds(aoi).filterDate('2019-01-01', '2019-12-31')
# check which bands are available in aoi in the most recent image (VV and VH are not available, maybe due to the direction the satellite is pointing)
print("PALSAR bands: " ,palsar.sort('system:time_start', False).first().clip(aoi).bandNames().getInfo()) 
palsar = palsar.select('HH')

# Sentinel-1 SAR data (split into two lines because it is very long)
sentinel = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate('2019-01-01', '2019-12-31')
# check which bands are available in aoi in the most recent image (HH and HV are not available, maybe due to the direction the satellite is pointing)
print("Sentinel-1 bands: " ,sentinel.sort('system:time_start', False).first().clip(aoi).bandNames().getInfo()) 
sentinel = sentinel.select('VV')
#sentinel = sentinel.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH')).filter(ee.Filter.eq('instrumentMode', 'IW')).select('HH').filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))



# Landsat 8 data: we only use 2019 data
landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterBounds(aoi).filterDate('2019-01-01', '2019-12-31')

# function to compute NDVI
def calculate_NDVI(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# compute NDVI
landsat_ndvi = landsat.map(calculate_NDVI).select('NDVI')




PALSAR bands:  ['HH', 'HV', 'angle', 'date', 'qa']
Sentinel-1 bands:  ['VV', 'VH', 'angle']


# Folium functions

In [5]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [6]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    #try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    #except:
    #    print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [7]:
#converts rgb value to hex
def rgb(r, g, b):
  return ('#%02x%02x%02x' % (r, g, b))

# Visualize the deforestation year map

In [40]:
# Get one image from the deforestation year collection, and remove values where no deforestation has occurred
deforestation_year_image = deforestation_year.first().clip(aoi)
deforestation_year_image = deforestation_year_image.updateMask(deforestation_year_image.neq(0))

# Convert to folium coordinates
location = aoi.centroid().coordinates().getInfo()[::-1]

# Colour parameters for deforestation_year
vis_params = {
  'min': 1982,
  'max': 2020,
  #"palette": ["#2892c7", "#b30800"]}
  "palette": [rgb(40,146,199), rgb(179,8,0)]}

# Create the map object.
f = folium.Figure(width=1000, height=500)
m = folium.Map(location=location, zoom_start=12).add_to(f)

# add satellite imagery
basemaps['Google Satellite Hybrid'].add_to(m)

# Add the deforestation year to the map
m.add_ee_layer(deforestation_year_image, vis_params, 'Deforestation Year')



# Get one image from the intensity collection
intensity_image = intensity.first().clip(aoi)

# Colour parameters for intensity
vis_params = {
  'min': 1,
  'max': 500,
  "palette": [rgb(0,128,0), rgb(255,0,0)]}

# Add the intensity to the map
m.add_ee_layer(intensity_image, vis_params, 'Intensity')




# Get one image from the transition map collection
transition_map_image = transition_map.clip(aoi)

# Colour parameters for transition map
vis_params = {
  "palette": [rgb(0,80,0), rgb(100,135,35), rgb(210,250,60 ), rgb(255,200,148), rgb(0,200,150 ), rgb(255,230,100 ), rgb(250,140,10 ), rgb(0,70,160 ), rgb(255,255,255)]}

# Add the transition map to the map
m.add_ee_layer(transition_map_image, vis_params, 'Transition Map')




# Get one image from the 2019 landsat NDVI collection with the least clouds
landsat_ndvi_image = landsat_ndvi.sort('CLOUD_COVER').first().clip(aoi) #2019 image is not blank, but when using .filterDate('2014-01-01', '2020-01-01') it was blank... why?

# Colour parameters for NDVI
vis_params = {
    'min': -1,
    'max': 1,
    "palette": [rgb(255, 255, 204), rgb(0, 153, 0)]}

# Add the NDVI to the map
m.add_ee_layer(landsat_ndvi_image, vis_params, 'Landsat NDVI')



# Get one image of the most recent PALSAR data
palsar_image = palsar.sort('system:time_start', False).first().clip(aoi)

# Colour parameters for PALSAR
vis_params = {
    'min': 0,
    'max': 10000,}

# Add the PALSAR to the map
m.add_ee_layer(palsar_image, vis_params, 'PALSAR')




# Get one image of the most recent Sentinel data
sentinel_image = sentinel.sort('system:time_start', False).first().clip(aoi)

# Colour parameters for Sentinel
vis_params = {
    'min': -50,
    'max': 1,}

# Add the Sentinel to the map
m.add_ee_layer(sentinel_image, vis_params, 'Sentinel-1')




# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
f